In [199]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from pymongo import MongoClient

import sys
sys.path.insert(0, '../src/features/')
sys.path.insert(0, '../src/data/')
sys.path.insert(0, '../src/visualization/flask_app/')

In [3]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

In [184]:
def book_cleaner(book_id):                                                                                         
    '''Finds book in mongo data base and returns a book without                                                    
    punctuation, numbers, and capital letters.'''                                                                  
                                                                                                                   
    book = collection.find({'_id': str(book_id)}, {'_id': 1, 'title': 1, 'author': 1, 'text': 1})                  
    book_text = []                                                                                                 
    book_title = ''                                                                                                
    for x in book:                                                                                                 
        book_text = x['text']                                                                                      
        book_title = x['title']                                                                                    
#     if len(book_text)>999999:                                                                                    
#         book_text = book_text[:999999]                                                                           
    if len(book_text)>50000:                                                                                       
        #beginning at 1000 to manually cut out the chapter words.                                                  
        book_text = book_text[1000:500000]                                                                         
    clean_book = re.sub('[”“]', '', book_text)                                                                     
    # remove numbers                                                                                               
    clean_book = re.sub('\w*\d\w*', ' ', clean_book)                                                               
    clean_book = re.sub('[cC][hH][aA][pP][tT][eE][rR]', ' ', clean_book)
    clean_book = re.sub('[tT][hH][yoOeE][\suUeE]', ' ', clean_book)
    clean_book = re.sub('[hH][a][ts][th]', ' ', clean_book)
    clean_book = re.sub('[dD][o][st][th]', ' ', clean_book)
    clean_book = re.sub('[Tt][h][i][n][e]', ' ', clean_book)
    clean_book = re.sub('[Ww][i][l][t]', ' ', clean_book)
    clean_book = re.sub('[Ww][h][i][l][s][t]', ' ', clean_book)
    clean_book = re.sub('[Oo][h]', ' ', clean_book)
    clean_book = re.sub('[yY][e]', ' ', clean_book)
    clean_book = re.sub('[nN][a][y]', ' ', clean_book)
    clean_book = re.sub('[\s][Yy][e][s][\s]', ' ', clean_book)
    clean_book = re.sub('[\s][Nn][a][y][\s]', ' ', clean_book)
    clean_book = re.sub('[\s][Ss][i][r][\s\.\,]', ' ', clean_book)
    clean_book = re.sub('[\s][eE][r][e][\s\.\,]', ' ', clean_book)


    print(book_title, len(book_text))                                                                              
    return clean_book                                                                                              
                         

In [185]:
list(collection.find({}, {'_id', 'title'}))

[{'_id': '1342', 'title': 'Pride and Prejudice'},
 {'_id': '84', 'title': 'Frankenstein'},
 {'_id': '219', 'title': 'Heart of Darkness'},
 {'_id': '1080', 'title': 'A Modest Proposal'},
 {'_id': '98', 'title': 'A Tale of Two Cities'},
 {'_id': '15399', 'title': 'The Importance of Being Earnest'},
 {'_id': '76', 'title': "Alice's Adventures in Wonderland"},
 {'_id': '2148', 'title': 'Moby Dick; or The Whale'},
 {'_id': '58915', 'title': 'Voyage to Jupiter'},
 {'_id': '160', 'title': 'The Adventures of Sherlock Holmes'},
 {'_id': '2097', 'title': 'The Souls of Black Folk'},
 {'_id': '5200', 'title': 'Metamorphosis'},
 {'_id': '1952', 'title': 'The Yellow Wallpaper'},
 {'_id': '58912', 'title': 'The Earth Quarter'},
 {'_id': '2600', 'title': 'War and Peace'},
 {'_id': '2591', 'title': "Grimms' Fairy Tales"},
 {'_id': '345', 'title': 'Dracula'},
 {'_id': '1260', 'title': 'Jane Eyre'},
 {'_id': '174', 'title': 'The Picture of Dorian Gray'},
 {'_id': '8800', 'title': 'Walden, and On The Duty

In [186]:
tales = collection.find({'title':'Legendary Tales of the Highlands (Volume 1 of 3)'}, {'text':1})

In [187]:
# tales = collection.find({'title':'The Merry Adventures of Robin Hood'}, {'text':1})

In [188]:
text = []
for x in tales:
    text = x['text']

In [189]:
str(text)

'LEGENDARYTALES OF THE HIGHLANDS.VOL. I.EDINBURGH: PRINTED BY T. CONSTABLE,\nPRINTER TO HER MAJESTY.[v]DEDICATION,       \nviiINTRODUCTION,       \nxiSTRATHDAWN,       \n1THE WATER-KELPIE’S BRIDLE AND THE MERMAID’S STONE,       \n13THE DOMINIE DEPARTS,       \n25HISTORY OF SERJEANT ARCHY STEWART,       \n32GALLANTRY OF THE SEVENTY-FIRST HIGHLAND LIGHT INFANTRY,       \n62LEGEND OF THE CLAN-ALLAN STEWARTS,       \n77FATE OF THE OULD AUNCIENT MONUMENTS,       \n261SUMMARY JUSTICE OF A HIGHLAND CHIEF,       \n113THE TABLES TURNED,       \n223\n[vii]My Dear Duke,The permission which you have so kindly given me to dedicate these Volumes to you, affords me a double source of gratification.In the first place, it recalls and strengthens the recollection of the first formation of that, which may now be called an old friendship between us; from the continuance of which I have, from time to time, derived so much valuable scientific and general information, as well as so much rational recreation\n

In [190]:
from spacy.lemmatizer import Lemmatizer                                                                                                                                                         
import spacy                                                                                                       

book = text
clean_book = book_cleaner(58913)                                                                                                                   
def spacy_lem_text(clean_book):                                                                                    
    '''uses spacy to create a lemmatized version of a text to feed into tf-idf.'''                                 
    nlp = spacy.load('en')                                                                                         
    spacy_book = nlp(clean_book)                                                                                   
    book_lem = ''                                                                                                  
    for token in spacy_book:                                                                                       
        if token.tag_ == 'NNP':                                                                                    
            x=0                                                                                                    
        else:                                                                                                      
            if token.tag_ != 'NNP' and token.lemma_ != '-PRON-':                                                   
                book_lem += ' %s' % token.lemma_                                                                   
    print(len(book_lem))                                                                                           
    return book_lem                                                                                                
                                                                                                                   
####################
robinh = spacy_lem_text(clean_book)


Legendary Tales of the Highlands (Volume 1 of 3) 312155
262110


In [191]:
with open('../src/models/tf_idf.pk', 'rb') as open_file:
    tfidf = pickle.load(open_file)

In [192]:
rh_tf = tfidf.transform([robinh])

In [193]:
words = tfidf.get_feature_names()

In [194]:
words[9871]

'look sister'

In [195]:
rh_array = rh_tf.toarray()
argmax_rh = rh_array.argsort()[-9:][::-1]

In [196]:
top_ten = argmax_rh[0][-9:]
top_ten

array([16379,   621,  5273,  6085,  8728, 16152,  7329, 13751,  1662])

In [197]:
for x in top_ten:
    print(words[x])
    print(rh_array[0][x])

stone
0.09817602308158212
arrow
0.10574043930790349
father
0.1071011160889987
gentleman
0.11597110364578293
lady
0.11997536319412816
st
0.1373661226610582
hill
0.18365108853430429
reply
0.18909015331854043
brother
0.2117327756149443


In [140]:
argmax_rh[0][1]


12110

In [141]:

for x in range(0,9):
    print(argmax_rh[0][x])
    print(words[argmax_rh[0][x]])
    

0
ab
12110
paralyse
12111
paralysis
12112
paramount
12113
parapet
12114
paraphrase
12115
parasol
12117
parched
12118
parchment


In [203]:
from get_dict_with_lems import get_lem_dict

In [204]:
lem_dict = get_lem_dict()

In [ ]:
lem_dict['32024']